# Код для парсинга параметров и стоимости автомобилей

### Цель программы
Создать тренировочный датасет, используя объявления с сайта auto.ru.

### Задачи:
1. Собрать данные;
2. Выгрузить датасет для дальнейшей обработки.

### 1. Импортируем библиотеки необходимые для обработки данных

In [ ]:
import numpy as np 
import pandas as pd 
import time
import requests as r
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

### 2. Выберем модель для сбора данных

#### 2.1. Выбор марки автомобилей

Если начать напрямую брать ссылки с https://auto.ru/moskva/cars/used/, то можно упереться в ограничения сайта на 99 страниц, поэтому нужно выбирать ссылки хитрее. Добавим в ссылку на сайт марку автомобиля.

Можно попытаться выбрать марку автомобиля с сайта:

    driver = webdriver.Firefox(executable_path=r'C:\Program Files\Mozilla Firefox\geckodriver.exe')
    START_URL = f'https://auto.ru/moskva/cars/used/'
    driver.get(START_URL)
    START_page = driver.execute_script("return document.body.innerHTML;")
    soup = BeautifulSoup(START_page, 'html.parser')
    for link in soup.findAll('a',{"class": "Link ListingPopularMMM__itemName"}):
        START_URL_brand.append(link['href'])
    driver.quit()
    
Но результат приносит только 7 самых популярных брендов.

Можно также скачать тестовый датасет:

In [ ]:
path = 'D:/Python/files/ML/Itog_7/' 
test = pd.read_csv(path+'test.csv')

Выбрать только нужные марки автомобелей из тестового датасета:

    Car_brand = list(test['brand'].unique())
    Car_type = [x.lower() for x in Car_brand]
    Car_type

В нашем случае марки автомобилей выбраны вручную:

In [ ]:
Car_type = ['bmw', 
            'mercedes', 
            'kia', 
            'hyundai', 
            'vaz', 
            'volkswagen', 
            'toyota', 
            'nissan',
            'ford',
            'audi',
            'skoda', 
            'mitsubishi', 
            'renault', 
            'mazda', 
            'chevrolet', 
            'opel',
            'land_rover', 
            'volvo', 
            'porsche',
            'peugeot', 
            'infiniti', 
            'honda', 
            'daewoo', 
            'gaz', 
            'citroen', 
            'lexus', 
            'subaru', 
            'suzuki', 
            'uaz', 
            'mini'
           ]

#### 2.2. Парсинг ссылок на объявления о продаже

Создадим список машин и список ссылок на объявления о продаже:

In [ ]:
cars_list = []
pages_url_list = []

Перебирая каждый выбранный брэнд 'Car_type' выбираем страницу с годом выпуска автомобиля 'j' и каждый год пролистываем 6 страниц 'i' для машин до 2005 года выпуска и 15 страниц 'i' - с 2005 года выпуска:

In [ ]:
driver = webdriver.Firefox(executable_path=r'C:\Program Files\Mozilla Firefox\geckodriver.exe')
for typ in Car_type:
    for j in range(1990,2022):
        START_URL = f'https://auto.ru/moskva/cars/'+typ+'/'+str(j)+'-year/used/'
        if j < 2005:
            n = 7
        else:
            n = 16
        for i in range(1,n):
            driver.get(START_URL+'?page='+str(i))
            if r.get(START_URL+'?page='+str(i)).status_code != 404:
                START_page = driver.execute_script("return document.body.innerHTML;")
                soup = BeautifulSoup(START_page, 'html.parser')
                for link in soup.findAll('a',{"class": "Link OfferThumb"}):
                    if link['href'] not in pages_url_list:
                        pages_url_list.append(link['href'])
                        time.sleep(0.2)
driver.quit()

Также посмотрим много ли в тестовом датасете машин до 90 года выпуска и самый старый автомобиль:

In [ ]:
(test['modelDate']<1990).value_counts()

In [ ]:
test['modelDate'].min()

Если в тестовом датасете есть автомобиль 1904 года перебираем каждый выбранный брэнд 'Car_type' по одной странице с годом выпуска с 1900 по 1990 в регионе 'Россия':

In [ ]:
driver = webdriver.Firefox(executable_path=r'C:\Program Files\Mozilla Firefox\geckodriver.exe')
for typ in Car_type:
    for j in range(1900,1990):
        START_URL = f'https://auto.ru/rossiya/cars/'+typ+'/'+str(j)+'-year/used/'
        driver.get(START_URL+'?page='+str(1))
        if r.get(START_URL+'?page='+str(1)).status_code != 404:
            START_page = driver.execute_script("return document.body.innerHTML;")
            soup = BeautifulSoup(START_page, 'html.parser')
            for link in soup.findAll('a',{"class": "Link OfferThumb"}):
                if link['href'] not in pages_url_list:
                    pages_url_list.append(link['href'])
                    time.sleep(0.2)
driver.quit()

Осматриваем полученные ссылки:

In [ ]:
Page_list = pd.DataFrame(pages_url_list)
Page_list

Запишем их в датасет:

In [ ]:
Page_list.to_csv(path+'page_list.csv', index=False)

Этот датасет пригодится на случай потери электроэнергии или интернета. Его всегда можно будет загрузить:

In [ ]:
#Page_list = pd.read_csv(path+'page_list.csv')

#### 2.3. Парсинг данных с объявлений

Ставим флаг, на случай потери соединения можно будет продолжить парсинг с флага. 

Тут можно задать свой флаг на случай восстановления:

In [ ]:
flag = 0

Смотрим flag и ссылку с который начнется (продолжится) парсинг:

In [ ]:
flag

In [ ]:
pages_url_list[flag]

С каждой ссылки пытаемся записать данные в cars_list:

In [ ]:
driver = webdriver.Firefox(executable_path=r'C:\Program Files\Mozilla Firefox\geckodriver.exe')
for item in pages_url_list: 
#for item in pages_url_list[flag:len(pages_url_list)]:
    driver.get(item)
    page = driver.execute_script("return document.body.innerHTML;")
    soup = BeautifulSoup(page, 'html.parser')
    response = r.get(item)
    response.encoding = 'utf8'
    soup_utf = BeautifulSoup(response.text, 'html.parser')
    flag = pages_url_list.index(item)
    try:
        settings_car = json.loads(soup_utf.find('script', type="application/ld+json").string)
        more_settings = [
        child.get_text(': ').replace('\xa0', ' ') for child in soup.find('ul',class_='CardInfo').children
        ]
        for i in range(len(more_settings)):
            settings_car[more_settings[i].split(':')[0]] = more_settings[i].split(':')[1]
        for key, value in settings_car['offers'].items():
            settings_car[key] = value
        cars_list.append(settings_car)
        print(item)
    except:
        pass
    time.sleep(0.2)
driver.quit()

### 3. Постобработка

Преобразуем полученные данные в DataFrame:

In [ ]:
cars_df = pd.DataFrame(cars_list)
cars_df

По уникальной ссылке 'url' избавимся от дубликатов в данных:

In [ ]:
cars_df = cars_df.drop_duplicates(subset=['url'])
cars_df.info()

Запишем полученные данные в файл:

In [ ]:
cars_df.to_csv(path+'cars_list.csv', index=False)

### Выводы:
1. На парсинг сайтов уходит очень много времени и нужен хороший стабильный интернет
2. Данные сохранены и выгружены датасет для дальнейшей обработки.
3. Через несколько дней можно загрузить ранее созданный 'Page_list' дополнить его и спарсить дополнительные данные через flag. 'Page_list' необходим, чтобы не парсить дубликаты благодаря строчке:

        if link['href'] not in pages_url_list: